In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/MyProject/RealFakeFaceImageClassification'

/content/drive/MyDrive/MyProject/RealFakeFaceImageClassification


In [3]:
!pwd

/content/drive/MyDrive/MyProject/RealFakeFaceImageClassification


### Import Libraries

In [ ]:
import os
import cv2 as cv
import numpy as np
from tensorflow.keras import utils

### Auto folder creation

In [ ]:
def create_dir(in_dir, out_dir, gdir_1, gdir_2):
    for label in os.listdir(path= in_dir):
        if not os.path.exists(os.path.join(out_dir, gdir_1, label)):
            os.makedirs(os.path.join(out_dir, gdir_1, label))

        if not os.path.exists(os.path.join(out_dir, gdir_2, label)):
            os.makedirs(os.path.join(out_dir, gdir_2, label))

### CannyEdgeDetector

In [ ]:
def preprocess_image(in_dir='input', out_dir='processed'):
    gdir_1 = 'canny_1'
    gdir_2 = 'canny_2'

    create_dir(in_dir, out_dir, gdir_1, gdir_2)

    for label in os.listdir(in_dir):
        print('Processing started for:', label)
        for image_name in os.listdir(os.path.join(in_dir, label)):
            if not image_name.endswith('.db'):
                in_path = os.path.join(in_dir, label, image_name)
                out_path_1 = os.path.join(out_dir, gdir_1, label, image_name)  # Path of Canny 1
                out_path_2 = os.path.join(out_dir, gdir_2, label, image_name)  # Path of Canny 2
                img = cv.imread(in_path, 0)  # Reading image in grayscale

                # Apply Canny Edge Detector with different thresholds
                canny_1 = cv.Canny(img, 100, 200)  # Applying Canny with lower thresholds
                cv.imwrite(out_path_1, canny_1)  # Writing filtered image to out_path_1

                canny_2 = cv.Canny(img, 200, 300)  # Applying Canny with higher thresholds
                cv.imwrite(out_path_2, canny_2)  # Writing filtered image to out_path_2

        print('------ Processing done ------')


In [ ]:
preprocess_image()

Processing started for: Real
------ Processing done ------
Processing started for: Fake
------ Processing done ------


In [ ]:
input_dir = os.path.join('processed', 'canny_2')
print('input_dir :', input_dir)

input_dir : processed/canny_2


In [ ]:
no_samples = 0
labels = os.listdir(input_dir)
print('Class labels : ', labels)

for label in labels:
    total = len(os.listdir(os.path.join(input_dir, label)))
    print(label, total)
    no_samples +=total
print('-'*30)
print('Total no of samples: ',no_samples)
print('-'*30)

Class labels :  ['Real', 'Fake']
Real 1081
Fake 960
------------------------------
Total no of samples:  2041
------------------------------


In [ ]:
img_rows = 128
img_cols = 128
channels = 3

### Converting the data into numpy files

In [ ]:
def load_data(input_dir, no_samples, labels):

    images = np.ndarray((no_samples, img_rows, img_cols, channels), dtype = np.float32)
    targets = np.zeros((no_samples,), dtype = np.uint8)

    i = 0
    print('-'*30)
    print('Loading training images...')
    print('-'*30)

    for j, label in enumerate(labels):
        image_names = os.listdir(os.path.join(input_dir, label))
        total = len(image_names)
        print(label, total)
        for image_name in image_names:
            img = cv.imread(os.path.join(input_dir, label, image_name), 1)
            img = np.array(cv.resize(img, (img_rows, img_cols)))

            images[i] = np.reshape(a= img, newshape= (img_rows, img_cols, channels))
            targets[i] = j

            i += 1
    print('Loading done.')

    targets = utils.to_categorical(targets, num_classes= len(labels))

    return images, targets

In [ ]:
X, Y = load_data(input_dir, no_samples, labels)

------------------------------
Loading training images...
------------------------------
Real 1081
Fake 960
Loading done.


In [ ]:
print(X.shape, Y.shape)

(2041, 128, 128, 3) (2041, 2)


In [ ]:
np.savez(file= 'dataset/data.npz', x= X, y= Y)

# Done